In [51]:
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
import datetime as dt
import pprint

In [52]:
api_key = '41dfefa12c616d5edf8dcd339324'
event_url = 'https://feeds.datagolf.com/historical-raw-data/event-list?file_format={file_format}&key={api_key}'
event_detail_url = 'https://feeds.datagolf.com/historical-raw-data/rounds?tour={tour}&event_id={event_id}&year={year}&file_format={file_format}&key={api_key}'
file_format = 'json'

In [53]:
events_request = requests.get(event_url.format(file_format=file_format,api_key=api_key))
events_json = events_request.json()
events = pd.DataFrame(events_json)
events.size

12502

In [54]:
event_details = []
pga_events = events.loc[events.tour == 'pga']
for event in pga_events.iterrows():
    row_data = event[1] # 1st val in tuple is index
    # index 0 = calendar year; 1 = date; 2 = event_id, 3 = event_name, 4 = sg_categoeries, 5 = tour, 6 = traditional stats 
    # update 11/28: parse series by column name not index
    _event_id = row_data['event_id'] 
    _year = row_data['calendar_year']
    _event_detail_url = event_detail_url.format(tour = 'pga', event_id = _event_id, year = _year, file_format=file_format, api_key=api_key)
    _event_detail = requests.get(_event_detail_url).json()
    event_details.append(_event_detail)

In [55]:
entries = []
for _event in event_details:
    _event_metadata = {key:_event[key] for key in _event.keys() if key != 'scores'}
    for _player in _event['scores']:
        _rounds_played = [x for x in _player.keys() if x[0:5] == 'round']
        _id = {key:_player[key] for key in _player.keys() if key not in _rounds_played}
        for n in _rounds_played:
            _round_data = _player[n]
            entry = {**_event_metadata,**_id,'round':n[-1],**_round_data}
            entries.append(entry)

In [63]:
data = pd.DataFrame(entries)
# data.to_csv('~/dev/sb/data/pga_stats.csv')
data